<b>CBERT TRAINING</b>  
Mulitlabel (~200 labels) classifier with distiliBert classify event membership of news items in BERT finetuning task

Trained on News Clustering corpus

In [7]:
!pip install simpletransformers
!pip install sentence_transformers
!pip install bert-tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 64 kB 2.3 MB/s 


In [8]:
from simpletransformers.classification import MultiLabelClassificationModel

import pandas as pd
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader
#import utils
import torch
print (torch.cuda.is_available())
print (torch.__version__)

True
1.12.0+cu113


In [9]:
from google.colab import drive
drive.mount('/content/drive')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
import sys
sys.path.insert(1, './drive/MyDrive/EventClustering/src')

In [11]:
#Define the model. Either from scratch of by loading a pre-trained model
model = SentenceTransformer('distilbert-base-nli-mean-tokens')

In [12]:
import corpus
lang = 'eng'
test_corpus = corpus.Corpus()
test_corpus.load_corpora(r"/drive/MyDrive/dataset/EventClustering/dataset.dev.json",
                        r"./drive/MyDrive/dataset/EventClustering/clustering.dev.json", set([lang]))

Pytorch Version: 1.12.0+cu113


ValueError: ignored

In [ ]:
input_df=pd.DataFrame(test_corpus.documents)
input_df=input_df[['text','event_id']]
print (input_df.head())

In [ ]:
input_df['text'] = input_df['text'].apply(lambda x: str(x).replace('\n', ' '))

input_df['text'] = input_df['text'].apply(lambda x: utils.preprocess_text(x))


In [ ]:
input_df.head()

In [ ]:
from sklearn.preprocessing import OneHotEncoder
# creating instance of one-hot-encoder
enc = OneHotEncoder(handle_unknown='ignore')

input_df['l_tmp'] =input_df['event_id'].astype('category')
input_df['labels_cat'] = input_df['l_tmp'].cat.codes
enc_df = pd.DataFrame(list(zip(enc.fit_transform(input_df[['labels_cat']]).toarray())), columns = ['labels'])
enc_df['text'] = input_df['text']
enc_df.head()


In [ ]:
sentences = (enc_df.text.values)
labels = (enc_df.labels.values)

In [ ]:
from sklearn.model_selection import train_test_split

train_df, eval_df = train_test_split(enc_df, random_state=2018, test_size=0.2)

In [ ]:
train_df = pd.DataFrame(train_df)
eval_df = pd.DataFrame(eval_df)

In [ ]:
train_df.shape

In [ ]:
from sentence_transformers import evaluation

def similarity_losstrain():
    #Define your train examples. You need more than just two examples...
    train_examples = [InputExample(texts=[raw_data['text_a'], raw_data['text_b']], label=raw_data['similarity'])]
    evaluator = evaluation.EmbeddingSimilarityEvaluator(sentences1=eval_df['text_a'], sentences2=eval_df['text_b'], scores=eval_df['similarity'], batch_size=32)
    #Define your train dataset, the dataloader and the train loss
    train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=32)
    train_loss = losses.BatchHardTripletLoss(model=model)

    #Tune the model
    model.fit(train_objectives=[(train_dataloader, train_loss)],  epochs=4, warmup_steps=100)

Create One-Hot vectors for the categorical labels to prepare for multilabel cluster fine tuning

In [ ]:
eval_df.shape

In [ ]:
output_dir ='models/BERT_ClusterMember_finetune'

In [ ]:
model_args = {
    "num_train_epochs": 2,
    "learning_rate": 2e-5,
    'reprocess_input_data': True,
    'train_batch_size':4, 
    'gradient_accumulation_steps':8, 
    'learning_rate': 2e-5, 
    'scheduler': "linear_schedule_with_warmup",
    'optimizer': 'AdamW',
    #'metric': metric,
    'use_multiprocessing': True,
    'tensorboard_dir': "tensorboard",
    'weight_decay':  0.02,
    'adam_epsilon':  2e-8,                                           
    'overwrite_output_dir': True, 
    'fp16': False,
    'max_seq_length': 128,
    'save_model_every_epoch': False,
    'output_dir': output_dir
}

In [ ]:
num_labels=input_df['event_id'].nunique()

In [ ]:
model = MultiLabelClassificationModel('roberta', 
                                      'roberta-base',  
                                      use_cuda=True, 
                                      num_labels=num_labels, 
                                      args=model_args
                                     )

In [ ]:
print (num_labels)

In [ ]:
model.train_model(train_df)

In [ ]:
enc_df.head()


In [ ]:
from transformers import BertTokenizer

# Load BERT tokenizer.
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [ ]:
print(' Original: ', sentences[0])
print('Tokenized: ', tokenizer.tokenize(sentences[0]))
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentences[0])))

In [ ]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []

# For every sentence...
for sent in sentences:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    encoded_sent = tokenizer.encode(
                        sent,                      
                        add_special_tokens = True, 
                      )
    # Add the encoded sentence to the list.
    input_ids.append(encoded_sent)

# Print sentence 0, now as a list of IDs.
print('Original: ', sentences[0])
print('Token IDs:', input_ids[0])

In [ ]:
print('Max sentence length: ', max([len(sen) for sen in input_ids]))


In [ ]:
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
MAX_LEN = 80
print('\nPadding/truncating all sentences to %d values...' % MAX_LEN)

print('\nPadding token: "{:}", ID: {:}'.format(tokenizer.pad_token, tokenizer.pad_token_id))

# Pad our input tokens with value 0.
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", 
                          value=0, truncating="post", padding="post")
print('\nDone.')

In [ ]:
# Create attention masks
attention_masks = []
for sent in input_ids:
    
    # Create the attention mask.
    #   - If a token ID is 0, then it's padding, set the mask to 0.
    #   - If a token ID is > 0, then it's a real token, set the mask to 1.
    att_mask = [int(token_id > 0) for token_id in sent]
    
    # Store the attention mask for this sentence.
    attention_masks.append(att_mask)

In [ ]:
from sklearn.model_selection import train_test_split
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, 
                                                            random_state=2018, test_size=0.2)

train_masks, validation_masks, _, _ = train_test_split(attention_masks, labels,
                                             random_state=2018, test_size=0.2)

In [ ]:
print (len(train_labels))


In [ ]:
train_labels = torch.from_numpy(np.ndarray(train_labels))


In [ ]:
# Convert all inputs and labels into torch tensors
import numpy as np
train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)

train_labels= torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)

train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

In [ ]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

# Load BertForSequenceClassification, the pretrained BERT model with a single 
# linear classification layer on top. 
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 2, 
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

model.cuda()

In [ ]:
params = list(model.named_parameters())
print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

In [ ]:
no_decay = ['bias', 'LayerNorm.weight']

# Separate the `weight` parameters from the `bias` parameters. 
# - For the `weight` parameters, this specifies a 'weight_decay_rate' of 0.01. 
# - For the `bias` parameters, the 'weight_decay_rate' is 0.0. 
optimizer_grouped_parameters = [
    # Filter for all parameters which *don't* include 'bias', 'gamma', 'beta'.
    {'params': [p for n, p in model.named_parameters()  if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.1},
    
    # Filter for parameters which *do* include those.
    {'params': [p for n, p in model.named_parameters()  if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]

# Adam optimizer
optimizer = AdamW(optimizer_grouped_parameters,
                  lr = 2e-5, # args.learning_rate 
                  eps = 1e-8 # args.adam_epsilon
                )


In [ ]:
from transformers import get_linear_schedule_with_warmup

epochs = 2

# Training steps = batches * number of epochs.
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [ ]:
import numpy as np

def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
import time
import datetime

def format_time(elapsed):
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
import random

# This training code is based on the `run_glue.py` 
# set the seed value everywhere
seed_val = 42
device='cuda'
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# Store the average loss after each epoch so we can plot them.
loss_values = []

for epoch_i in range(0, epochs):

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_loss = 0

    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))
            
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)

        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        # Always clear any previously calculated gradients before performing a
        # backward pass. 
        model.zero_grad()        
        print ("THE INPUT IDS: ",b_input_ids.long())
        print ("ATTENTION MASK: ",b_input_mask.long())
        print ("LABELS: ", b_labels.long())
        # Perform a forward pass (evaluate the model on this training batch).
        # This will return the loss 
        outputs = model(b_input_ids.long(), 
                    token_type_ids=None, 
                    attention_mask=b_input_mask, 
                    labels=b_labels.long())
        
        # pull the loss value out of the tuple.
        loss = outputs[0]

        # Accumulate the training loss over all of the batches .
        total_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Stop exploding gradients, Clip the norm of the gradients to 1.0.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over the training data.
    avg_train_loss = total_loss / len(train_dataloader)            
    
    # Store the loss value for plotting the learning curve.
    loss_values.append(avg_train_loss)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    #  Validation Step

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)
        
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch

        # Do not compute or store gradients to save space
        with torch.no_grad():        
  
            # Forward pass, calculate logit predictions.
            outputs = model(b_input_ids.long(), 
                            token_type_ids=None, 
                            attention_mask=b_input_mask.long())
        
        # Get the "logits" output by the model.
        logits = outputs[0]
        print ("LOGITS ARE: ", logits)
        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # Calculate the accuracy for this batch of test sentences.
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        
        # Accumulate the total accuracy.
        eval_accuracy += tmp_eval_accuracy

        # Track the number of batches
        nb_eval_steps += 1

    # Report the final accuracy for this validation run.
    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns

# Use plot styling from seaborn.
sns.set(style='darkgrid')

# Increase the plot size and font size.
sns.set(font_scale=1.5)
plt.rcParams["figure.figsize"] = (12,6)

# Plot the learning curve.
plt.plot(loss_values, 'b-o')

# Label the plot.
plt.title("Training loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")

plt.show()

In [ ]:
import os

# Saving best-practices: if you use defaults names for the model, you can reload it using from_pretrained()

output_dir = './outputs_man/'

# Create output directory if needed
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print("Saving model to %s" % output_dir)

# Save a trained model, configuration and tokenizer using `save_pretrained()`.
# They can then be reloaded using `from_pretrained()`
model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
model_to_save.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)